# Getting V2M data from NASA

In [1]:
import os
import re
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from calendar import monthrange
from opendap_download.multi_processing_download import DownloadManager

In [2]:
####### INPUTS - CHANGE THESE #########
username = 'bhauma' # Username for MERRA download account
password = 'Ankita@0123' # Password for MERRA download account

In [3]:
years = [2020] # List of years for which data will be downloaded
field_id = 'V2M' # ID of field in MERRA-2 - find ID here: https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf 
field_name = '2M Northward Wind' # Name of field to be stored with downloaded data (can use any name you like)
database_name = 'M2TMNXSLV' # Name of database in which field is stored, can be looked up by ID here: 
database_id = 'tavgM_2d_slv_Nx'

In [4]:
df = pd.read_csv("../Substation/geo_locations/geo_location_FL.csv")
df['name'] = 'Florida'

In [5]:
df

,lon,lat,name
0,-83.550482,30.052997,Florida
1,-83.545344,30.061611,Florida
2,-83.169863,30.125400,Florida
3,-83.169966,30.124813,Florida
4,-82.934986,29.999176,Florida
...,...,...,...
1978,-87.333326,30.462818,Florida
1979,-87.304204,30.470746,Florida
1980,-83.176040,30.374848,Florida
1981,-83.180835,30.442726,Florida


In [6]:
locs = df.values.tolist()
len(locs)

1983

In [7]:
aggregator = 'max' # Method by which data will be aggregated over days and weeks. Can be "sum", "mean", "min", or "max" 

In [8]:
lat_coords = np.arange(0, 361, dtype=int)
lon_coords = np.arange(0, 576, dtype=int)
#database_url = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2_MONTHLY/' + database_name + '.5.12.4/'
database_url = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2_MONTHLY/M2TMNXSLV.5.12.4/'
NUMBER_OF_CONNECTIONS = 5

In [9]:
####### DOWNLOAD DATA #########
# Translate lat/lon into coordinates that MERRA-2 understands
def translate_lat_to_geos5_native(latitude):
    """
    The source for this formula is in the MERRA2 
    Variable Details - File specifications for GEOS pdf file.
    The Grid in the documentation has points from 1 to 361 and 1 to 576.
    The MERRA-2 Portal uses 0 to 360 and 0 to 575.
    latitude: float Needs +/- instead of N/S
    """
    return ((latitude + 90) / 0.5)

In [10]:
def translate_lon_to_geos5_native(longitude):
    """See function above"""
    return ((longitude + 180) / 0.625)

In [11]:
def find_closest_coordinate(calc_coord, coord_array):
    """
    Since the resolution of the grid is 0.5 x 0.625, the 'real world'
    coordinates will not be matched 100% correctly. This function matches 
    the coordinates as close as possible. 
    """
    # np.argmin() finds the smallest value in an array and returns its
    # index. np.abs() returns the absolute value of each item of an array.
    # To summarize, the function finds the difference closest to 0 and returns 
    # its index. 
    index = np.abs(coord_array-calc_coord).argmin()
    return coord_array[index]

In [12]:
def translate_year_to_file_number(year):
    """
    The file names consist of a number and a meta data string. 
    The number changes over the years. 1980 until 1991 it is 100, 
    1992 until 2000 it is 200, 2001 until 2010 it is  300 
    and from 2011 until now it is 400.
    """
    file_number = ''
    
    if year >= 1980 and year < 1992:
        file_number = '100'
    elif year >= 1992 and year < 2001:
        file_number = '200'
    elif year >= 2001 and year < 2011:
        file_number = '300'
    elif year >= 2011:
        file_number = '400'
    else:
        raise Exception('The specified year is out of range.')
    return file_number

In [13]:
def generate_url_params(parameter, time_para, lat_para, lon_para):
    """Creates a string containing all the parameters in query form"""
    parameter = map(lambda x: x + time_para, parameter)
    parameter = map(lambda x: x + lat_para, parameter)
    parameter = map(lambda x: x + lon_para, parameter)
    return ','.join(parameter)

In [ ]:
def generate_download_links(download_years, base_url, dataset_name, url_params):
    """
    Generates the links for the download. 
    download_years: The years you want to download as array. 
    dataset_name: The name of the data set. For example tavg1_2d_slv_Nx
    """
    urls = []
    for y in download_years: 
        y_str = str(y)
        file_num = translate_year_to_file_number(y)
        for m in range(1,13):
            m_str = str(m).zfill(2)
            # Create the file name string
            file_name = 'MERRA2_{num}.{name}.{y}{m}.nc4'.format(
                num=file_num, name=dataset_name, 
                y=y_str, m=m_str)
            # Create the query
            query = '{base}{y}/{name}.nc4?{params}'.format(
                base=base_url, y=y_str,
                name=file_name, params=url_params)
            urls.append(query)
    return urls

In [ ]:
print('DOWNLOADING DATA FROM MERRA')
print('Predicted time: ' + str(len(years)*len(locs)*6) + ' minutes')
print('=====================')
for lat, lon, loc in locs:
    print('Downloading ' + field_name + ' data for ' + loc + str(lat) + str(lon))
    # Translate the coordinates that define your area to grid coordinates.
    lat_coord = translate_lat_to_geos5_native(lat)
    lon_coord = translate_lon_to_geos5_native(lon)
    # Find the closest coordinate in the grid.
    lat_closest = find_closest_coordinate(lat_coord, lat_coords)
    lon_closest = find_closest_coordinate(lon_coord, lon_coords)
    # Generate URLs for scraping
    requested_lat = '[{lat}:1:{lat}]'.format(lat=lat_closest)
    requested_lon = '[{lon}:1:{lon}]'.format(lon=lon_closest)
    parameter = generate_url_params([field_id], '[0:1:0]', requested_lat, requested_lon)
    generated_URL = generate_download_links(years, database_url, database_id, parameter)
    download_manager = DownloadManager()
    download_manager.set_username_and_password(username, password)
    download_manager.download_path = field_name + '/' + loc
    download_manager.download_urls = generated_URL
    %time download_manager.start_download(NUMBER_OF_CONNECTIONS)

In [14]:
######### OPEN, CLEAN, MERGE, MERGE DATA AND WRITE CSVS ##########
def extract_date(data_set):
    """
    Extracts the date from the filename before merging the datasets. 
    """ 
    if 'HDF5_GLOBAL.Filename' in data_set.attrs:
        f_name = data_set.attrs['HDF5_GLOBAL.Filename']
    elif 'Filename' in data_set.attrs:
        f_name = data_set.attrs['Filename']
    else: 
        raise AttributeError('The attribute name has changed again!')
    # find a match between "." and ".nc4" that does not have "." .
    exp = r'(?<=\.)[^\.]*(?=\.nc4)'
    res = re.search(exp, f_name).group(0)
    # Extract the date. 
    y, m = res[0:4], res[4:6]
    date_str = ('%s-%s' % (y, m))
    data_set = data_set.assign(date=date_str)
    return data_set

In [15]:
# Open nc4 files as dataframes, perform aggregations and save as CSV files
print('CLEANING AND MERGING DATA')
print('Predicted time: ' + str(len(years)*len(locs)*0.1) + ' minutes')
print('=====================')

lats = []
lons = []
dates = []
v2ms = []

for lat, lon, loc in locs:
    print('Cleaning and merging ' + field_name + ' data for ' + loc + str(lat) + str(lon))
    for file in os.listdir(field_name + '/' + loc):
        if '.nc4' in file:
            try:
                with xr.open_mfdataset(field_name + '/' + loc + '/' + file, preprocess=extract_date) as df:
                    v2ms.append(df.to_dataframe()["V2M"].values[0])
                    dates.append(df.to_dataframe()["date"].values[0])
                    lats.append(lat)
                    lons.append(lon)
            except:
                print('Issue with file ' + file)
    
df_monthly = pd.DataFrame({"date": dates, "lat": lats, "lon": lons, "V2M": v2ms})
df_monthly.to_csv(field_name + '/' + loc + '_monthly.csv', index = False)

print('FINISHED')

CLEANING AND MERGING DATA
Predicted time: 198.3 minutes
Cleaning and merging 2M Northward Wind data for Florida-83.55048166830.0529967590001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.54534396630.0616113
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.16986348230.1254002360001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.1699664630.1248128030001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.934986390999929.9991755300001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.935429564999929.998869175
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.1829121130.371626549
Issue with file MERRA2_400.tavgM_

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.17789085229.2676236670001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-85.94425543130.6202143940001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-85.643222594999930.3260808960001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.58183101628.0712166770001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.61133538628.018766242
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-85.724250078999930.8034863940001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-85.94646086330.733262202
Issue with file MERRA2_400.

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.63170087130.105270692
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.56990567730.160349877
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.71037092830.217664079
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.72754899330.29322412
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.13576435829.6167315880001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.776271156999928.0227805710001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.7687191427.9996538820001
Issue with file MERRA2_400.tavgM_2d_slv_N

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.36829004528.0658721830001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.358385031999927.984988269
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.441571704999927.9252894210001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.733100417999928.6088687220001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.3898960328.6697616170001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.27742847427.9728499770001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.21860934328.0256023920001
Issue with file MERR

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.41729218227.4305786780001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.441081477999927.325250528
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.54587681527.603359551
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.48253538727.483893955
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.60203130528.8521108300001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-80.375302104999926.6936874570001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-80.37613245426.702926651
Issue with file MERRA2_400.tavgM_2d

...

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.72887058628.017243798
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.69725821127.9670802540001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.75183936926.1761787560001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.74302225726.140464566
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.703862662999926.0359760630001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.70142731326.036803954
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.688061027999926.2026011680001
Issue with file MERRA2_400.tavg

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.612662730999928.0919557920001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.62619801230.3629829980001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.608061947999930.36658851
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.60064871230.366562979
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.60757996427.961299409
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.59922212527.909984783
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.63998013730.3293592090001
Issue with file MERRA2_400.tavgM_2d_

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.89467497629.526210501
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.8480139429.4962710300001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.939510509999930.276419784
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.78003574829.9531483430001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-82.97828219930.2931167450001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.64145561228.214535019
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.66928085330.3330010510001
Issue with file MERRA2_400.tavgM_2d_

Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.52221324.659792
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-80.416654999999927.212188
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.91347427.8372600000001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-80.682728.4833340000001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-80.445784999999925.513472
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-81.90437527.8988180000001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-87.248913387999930.599732599
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4


Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.180835424999930.4427256720001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
Cleaning and merging 2M Northward Wind data for Florida-83.18183965430.4428362290001
Issue with file MERRA2_400.tavgM_2d_slv_Nx.202009.nc4
FINISHED
